In [1]:
import SimpleITK as sitk

T1_path = "../IXI_dataset/IXI_registrated/IXI230-IOP-0869/T1.nii.gz"
T2_path = "../IXI_dataset/IXI_registrated/IXI230-IOP-0869/T2.nii.gz"

moving_image = sitk.ReadImage(str(T1_path), sitk.sitkFloat32)
fixed_image = sitk.ReadImage(str(T2_path), sitk.sitkFloat32)

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
from ipywidgets import interact,fixed

def display_images(fixed_image_z, moving_image_z, fixed_npa, moving_npa):
    # Create a figure with two subplots and the specified size.
    plt.subplots(1, 2, figsize=(10, 8))

    # Draw the fixed image in the first subplot.
    plt.subplot(1, 2, 1)
    plt.imshow(fixed_npa[fixed_image_z, :, :], cmap=plt.cm.Greys_r)
    plt.title("fixed image")
    plt.axis("off")

    # Draw the moving image in the second subplot.
    plt.subplot(1, 2, 2)
    plt.imshow(moving_npa[moving_image_z, :, :], cmap=plt.cm.Greys_r)
    plt.title("moving image")
    plt.axis("off")

    plt.show()


interact(
    display_images,
    fixed_image_z=(0, fixed_image.GetSize()[2] - 1),
    moving_image_z=(0, moving_image.GetSize()[2] - 1),
    fixed_npa=fixed(sitk.GetArrayViewFromImage(fixed_image)),
    moving_npa=fixed(sitk.GetArrayViewFromImage(moving_image)),
);


interactive(children=(IntSlider(value=86, description='fixed_image_z', max=173), IntSlider(value=93, descripti…

In [5]:
import numpy as np
def display_images_with_alpha(image_z, alpha, fixed, moving):
    # img = (1.0 - alpha) * fixed[:, :, image_z] + alpha * moving[:, :, image_z]
    # plt.imshow(sitk.GetArrayViewFromImage(img), cmap=plt.cm.Greys_r)
    # plt.axis("off")
    # plt.show()
    # 创建 RGB 图像
    # fixed_slice = fixed[:, :, image_z]
    # moving_slice = moving[:, :, image_z]
    
    # 将 fixed 映射到绿色通道，moving 映射到红色通道
    shape = sitk.GetArrayViewFromImage(fixed[:, :, image_z]).shape
    
    fixed_slice = sitk.GetArrayFromImage(fixed[:, :, image_z])
    moving_slice = sitk.GetArrayFromImage(moving[:, :, image_z])
    # 归一化
    fixed_slice = fixed_slice / np.max(fixed_slice)
    moving_slice = moving_slice / np.max(moving_slice)
    
    img = np.zeros((shape[0], shape[1], 3))
    img[:, :, 1] = (1.0 - alpha) * fixed_slice  # 绿色通道
    img[:, :, 0] = alpha * moving_slice         # 红色通道
    
    plt.imshow(img)
    plt.axis("off")
    plt.show()

initial_transform=sitk.Transform(3, sitk.sitkIdentity)

# initial_transform = sitk.CenteredTransformInitializer(
#     fixed_image,
#     moving_image,
#     sitk.Euler3DTransform(),
#     sitk.CenteredTransformInitializerFilter.GEOMETRY,
# )

moving_resampled = sitk.Resample(
    moving_image,
    fixed_image,
    initial_transform,
    sitk.sitkLinear,
    0.0,
    moving_image.GetPixelID(),
)

interact(
    display_images_with_alpha,
    image_z=(0, fixed_image.GetSize()[2] - 1),
    alpha=(0.0, 1.0, 0.05),
    fixed=fixed(fixed_image),
    moving=fixed(moving_resampled),
);

interactive(children=(IntSlider(value=86, description='image_z', max=173), FloatSlider(value=0.5, description=…

In [ ]:
# initial_transform = sitk.CenteredTransformInitializer(
#     fixed_image,
#     moving_image,
#     sitk.Euler3DTransform(),
#     sitk.CenteredTransformInitializerFilter.GEOMETRY,
# )

# moving_resampled2 = sitk.Resample(
#     moving_resampled,
#     fixed_image,
#     initial_transform,
#     sitk.sitkLinear,
#     0.0,
#     moving_resampled.GetPixelID(),
# )

# interact(
#     display_images_with_alpha,
#     image_z=(0, fixed_image.GetSize()[2] - 1),
#     alpha=(0.0, 1.0, 0.05),
#     fixed=fixed(fixed_image),
#     moving=fixed(moving_resampled2),
# );

In [ ]:
#NMI (custom)
import numpy as np
from sklearn import metrics
def mutual_information(moving_image, fixed_image, nbins=64):
    """
    计算两幅图像之间的归一化互信息 (NMI)。
    
    参数:
      moving_image: SimpleITK.Image, moving image
      fixed_image: SimpleITK.Image, fixed image
      nbins: 用于直方图统计的bin数量，默认64
      
    返回:
      nmi: 归一化互信息
    """
    # 将SimpleITK图像转换为numpy数组
    moving_arr = sitk.GetArrayFromImage(moving_image)
    fixed_arr  = sitk.GetArrayFromImage(fixed_image)
    
    # 获取最小层数
    slice_min = min(moving_image.GetSize()[-1], fixed_image.GetSize()[-1])

    # 按照 [z,y,x] 排序
    moving_arr = moving_arr[:slice_min, :, :].ravel()
    fixed_arr  = fixed_arr[:slice_min, :, :].ravel()
    
    # 计算联合直方图
    joint_hist, moving_edges, fixed_edges = np.histogram2d(moving_arr, fixed_arr, bins=nbins)
    
    # 计算联合熵
    joint_prob = joint_hist / np.sum(joint_hist)  # 归一化联合直方图
    joint_entropy = -np.sum(joint_prob * np.log(joint_prob + 1e-10))  # 加一个小常数避免对数为零

    # 计算固定图像的边缘熵
    fixed_prob = np.sum(joint_prob, axis=0)  # 对moving图像维度求和，得到fixed图像的边缘分布
    fixed_entropy = -np.sum(fixed_prob * np.log(fixed_prob + 1e-10))  # 加一个小常数避免对数为零

    # 计算移动图像的边缘熵
    moving_prob = np.sum(joint_prob, axis=1)  # 对fixed图像维度求和，得到moving图像的边缘分布
    moving_entropy = -np.sum(moving_prob * np.log(moving_prob + 1e-10))  # 加一个小常数避免对数为零

    # 计算互信息
    mutual_info = fixed_entropy + moving_entropy - joint_entropy

    # 归一化互信息
    nmi = mutual_info / np.sqrt(fixed_entropy * moving_entropy)
    return mutual_info,nmi

def normalized_mutual_information(moving_image, fixed_image, nbins=64):
    
    """
    计算两幅图像之间的归一化互信息 (NMI)。
    
    参数:
      moving_image: SimpleITK.Image, moving image
      fixed_image: SimpleITK.Image, fixed image
      nbins: 用于直方图统计的bin数量，默认64
      
    返回:
      nmi: 归一化互信息
    """
    # 将SimpleITK图像转换为numpy数组
    moving_arr = sitk.GetArrayFromImage(moving_image)
    fixed_arr  = sitk.GetArrayFromImage(fixed_image)
    
    # 获取最小层数
    slice_min = min(moving_image.GetSize()[-1], fixed_image.GetSize()[-1])

    # 按照 [z,y,x] 排序
    moving_arr = moving_arr[:slice_min, :, :].ravel()
    fixed_arr  = fixed_arr[:slice_min, :, :].ravel()
    
    # 计算联合直方图
    joint_hist, moving_edges, fixed_edges = np.histogram2d(moving_arr, fixed_arr, bins=nbins)
    
    # # 归一化得到联合概率分布
    # joint_prob = joint_hist / np.sum(joint_hist)

    # # 根据联合直方图计算边缘直方图（概率分布）
    # moving_prob = np.sum(joint_prob, axis=1)
    # fixed_prob  = np.sum(joint_prob, axis=0)
    
    # # 定义一个小数值避免log(0)的问题
    # eps = np.finfo(float).eps

    # # 计算熵值（只对大于0的概率进行计算）
    # H_moving = - np.sum(moving_prob[moving_prob > 0] * np.log(moving_prob[moving_prob > 0] + eps))
    # H_fixed  = - np.sum(fixed_prob[fixed_prob > 0]  * np.log(fixed_prob[fixed_prob > 0] + eps))
    # H_joint  = - np.sum(joint_prob[joint_prob > 0]  * np.log(joint_prob[joint_prob > 0] + eps))
    
    # # 归一化互信息
    # nmi = (H_moving + H_fixed) / H_joint
    # return nmi
    pxy = joint_hist / float(np.sum(joint_hist))
    px = np.sum(pxy, axis=1) # marginal for x over y
    py = np.sum(pxy, axis=0) # marginal for y over x
    # 计算边缘熵 Hx 与 Hy
    # 注意：对所有概率大于0的项计算熵，避免 log(0) 出现问题
    Hx = -np.sum(px[px > 0] * np.log(px[px > 0]))
    Hy = -np.sum(py[py > 0] * np.log(py[py > 0]))

    # 计算联合熵 Hxy，只对非零项计算
    nzs = pxy > 0
    Hxy = -np.sum(pxy[nzs] * np.log(pxy[nzs]))

    # 使用 NMI 定义之一，这里采用 (Hx + Hy) / Hxy
    nmi = (Hx + Hy) / Hxy
    return nmi
print("NMI1: ", mutual_information(moving_image, fixed_image))
print("NMI2: ", mutual_information(moving_resampled, fixed_image))
print("NMI3: ", mutual_information(fixed_image, fixed_image))

# registration_method = sitk.ImageRegistrationMethod()
# registration_method.GetMetricSamplingPercentagePerLevel()


NMI1:  (0.2786630792864804, 0.12845533266763437)
NMI2:  (0.5667728137196146, 0.28220584681996363)
NMI3:  (1.8711607411185749, 0.9999999999999999)


In [40]:
#NMI (sklearn)
#NMI (custom)
import numpy as np
from sklearn.metrics import normalized_mutual_info_score

def normalized_mutual_information(moving_image, fixed_image, nbins=50):
    """
    计算两幅图像之间的归一化互信息 (NMI)。
    
    参数:
      moving_image: SimpleITK.Image, moving image
      fixed_image: SimpleITK.Image, fixed image
      nbins: 用于直方图统计的bin数量，默认64
      
    返回:
      nmi: 归一化互信息
    """
    # 将SimpleITK图像转换为numpy数组
    moving_arr = sitk.GetArrayFromImage(moving_image)
    fixed_arr  = sitk.GetArrayFromImage(fixed_image)
    
    # 获取最小层数
    slice_min = min(moving_image.GetSize()[-1], fixed_image.GetSize()[-1])
    print("slice_min: ", slice_min)
    print("moving_arr.shape: ", moving_arr.shape)

    # 按照 [z,y,x] 排序
    moving_arr = moving_arr[:slice_min, :, :].ravel()
    fixed_arr  = fixed_arr[:slice_min, :, :].ravel()
    
    nmi = normalized_mutual_info_score(
        moving_arr,
        fixed_arr,
        average_method='arithmetic',
    )
    return nmi

print("NMI1: ", normalized_mutual_information(moving_image, fixed_image))
print("NMI2: ", normalized_mutual_information(moving_resampled, fixed_image))

slice_min:  174
moving_arr.shape:  (187, 256, 256)


/home/czfy/.conda/envs/czfy310/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


NMI1:  0.9514546680167647
slice_min:  174
moving_arr.shape:  (174, 256, 256)


/home/czfy/.conda/envs/czfy310/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:59: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


NMI2:  0.8656445738017253


In [ ]:
registration = sitk.ImageRegistrationMethod()
registration.sampl

AttributeError: 'ImageRegistrationMethod' object has no attribute 'MetricSamplingStrategy'

In [ ]:
print(moving_image.GetSize())
print(moving_resampled.GetSize())

(256, 256, 187)
(256, 256, 174)


In [ ]:
def g(x, y):
    print(f"x = {x}  and y = {y}")
interact(g, x=(0, 10, 1), y=(0, 100, 5))

interactive(children=(IntSlider(value=5, description='x', max=10), IntSlider(value=50, description='y', step=5…

<function __main__.g(x, y)>